In [ ]:
! pip install transformers torch

In [ ]:
! pip install scikit-learn

In [ ]:
import json
import torch
import pandas as pd
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments, pipeline
import random

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Дообучение модели

In [ ]:
def expand_mask(text, target_word, tokenizer):
    target_tokens = tokenizer.tokenize(target_word)
    num_target_tokens = len(target_tokens)

    if "[MASK]" in text:
        return text.replace("[MASK]", " ".join(["[MASK]"] * num_target_tokens), 1)
    else:
        raise ValueError(f"Ошибка: в тексте нет [MASK] → {text}")

In [ ]:
class SynonymDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["context"]
        target_word = item["word"]

        if "[MASK]" not in text:
            raise ValueError(f"Ошибка: в тексте нет [MASK] → {text}")

        text = expand_mask(text, target_word, self.tokenizer)
        encoding = self.tokenizer(
            text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )

        labels = torch.full_like(encoding["input_ids"], -100)

        mask_idx = (encoding["input_ids"] == self.tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
        target_ids = self.tokenizer.encode(target_word, add_special_tokens=False)

        if len(mask_idx) != len(target_ids):
            raise ValueError(f"Ошибка маскировки: {len(mask_idx)} масок, но '{target_word}' = {len(target_ids)} токенов")

        labels[0, mask_idx] = torch.tensor(target_ids)

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": labels.squeeze()
        }

In [ ]:
def train_bert(json_file, model_name="KoichiYasuoka/roberta-small-belarusian", output_dir="./bert-synonyms"):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    dataset = SynonymDataset(data, tokenizer)
    model = AutoModelForMaskedLM.from_pretrained(model_name)

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="no",
        save_strategy="steps",
        save_steps=500,
        logging_dir="./logs",
        logging_steps=100,
        num_train_epochs=6,
        per_device_train_batch_size=16,
        learning_rate=5e-5,
        report_to="none"
    )

    trainer = Trainer(model=model, args=training_args, train_dataset=dataset)
    trainer.train()

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)


train_bert("train_data.json")

In [ ]:
! pip install stanza

In [ ]:
import stanza

nlp = stanza.Pipeline(lang='be', processors='tokenize,pos,lemma')

In [ ]:
def lemmatize(word):
    doc = nlp(word)
    for sentence in doc.sentences:
        for token in sentence.tokens:
            return token.words[0].lemma
    return word

# Загрузка словаря

In [ ]:
from collections import defaultdict

def load_synonyms_from_excel(file_path):
    df = pd.read_excel(file_path)
    synonym_dict = defaultdict(list)
    for _, row in df.iterrows():
        word = str(row[0]).strip()
        lemma = lemmatize(word)
        synonyms = str(row[1]).split(',')
        synonym_dict[lemma].extend(s.strip() for s in synonyms if s.strip())
    return synonym_dict

In [ ]:
synonym_dict = load_synonyms_from_excel("synonyms_table_result.xlsx")

# Ранжирование предсказаний модели

In [ ]:
def find_masked_index(tokens):
    try:
        return tokens.index('[MASK]')
    except ValueError:
        raise ValueError("Токен [MASK] не найден в списке токенов.")


# --- Получение предсказаний и добавление синонимов из словаря ---
def get_masked_predictions(sentence, original_word, synonym_dict=None, top_k=100):
    encoding = tokenizer.encode(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(encoding[0])

    masked_index = find_masked_index(tokens)

    with torch.no_grad():
        outputs = model(input_ids=encoding)
        predictions = outputs.logits

    predicted_ids = torch.topk(predictions[0, masked_index], top_k).indices.numpy()
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_ids)

    filtered_words = [word for word in predicted_tokens if word.isalpha() and len(word) >= 3]

    if synonym_dict:
        lemma = lemmatize(original_word)
        if lemma in synonym_dict:
            extra_synonyms = synonym_dict[lemma]
            filtered_words = list(set(filtered_words + extra_synonyms))

    return filtered_words


# --- Ранжирование по сходству предложений ---
def rank_by_similarity(original_sentence, predicted_tokens, original_word):
    original_sentence_with_original_word = original_sentence.replace('[MASK]', original_word)
    original_vector = sentence_model.encode(original_sentence_with_original_word)

    sentences_with_predictions = [
        original_sentence.replace('[MASK]', token) for token in predicted_tokens
    ]
    prediction_vectors = sentence_model.encode(sentences_with_predictions)

    similarities = cosine_similarity(prediction_vectors, [original_vector])

    ranked_suggestions = sorted(
        zip(predicted_tokens, similarities[:, 0]),
        key=lambda x: x[1],
        reverse=True
    )

    return ranked_suggestions


model_path = "./bert-synonyms"
tokenizer = AutoTokenizer.from_pretrained("./bert-synonyms")
model = AutoModelForMaskedLM.from_pretrained(model_path)
sentence_model = SentenceTransformer("sentence-transformers/LaBSE")

In [ ]:
synonym_dict

In [ ]:
sentence = "Была ў Фёдара яшчэ адна рэдкая [MASK]: па сваёй уласнай ініцыятыве ён ніколі нікога не падвозіў, не падбіраў."
original_word = "асаблівасць"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

асаблівасць: 1.0000
адметнасць: 0.9986
характарыстыка: 0.9974
уласцівасць: 0.9972
своеасаблівасць: 0.9961
характар: 0.9927
спецыфіка: 0.9919
атрыбут: 0.9912
прыкмета: 0.9900
рыса: 0.9897


In [ ]:
sentence = "Жанчына была ўпэўненая ў тым, хто віноўнік гэтай [MASK]."
original_word = "бяды"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

бяды: 1.0000
няшчасце: 0.9825
гора: 0.9773
праблемы: 0.9764
нягода: 0.9747
нядоля: 0.9712
бяздолле: 0.9659
зло: 0.9618
справы: 0.9615
справай: 0.9599


In [ ]:
sentence = "Але ўсё гэта можа адбыцца толькі тады, калі нашы [MASK] збудуцца."
original_word = "прадбачанні"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

прадгаданне: 0.9848
прадказанне: 0.9828
прароцтва: 0.9797
прагноз: 0.9695
пажаданні: 0.9641
чаканні: 0.9625
просьбы: 0.9506
жаданні: 0.9473
мары: 0.9471
прапановы: 0.9435


In [ ]:
sentence = "Пытанні, прыспешваючы, падганяючы адно другое, адгучалі, і ў пакоі надоўга ўсталявалася [MASK]."
original_word = "цішыня"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

ціша: 0.9924
ціш: 0.9812
супакой: 0.9797
спакой: 0.9782
спакойлівасць: 0.9759
бязгучнасць: 0.9696
галасаванне: 0.9628
хваляванне: 0.9607
мяжа: 0.9547
пуста: 0.9530


In [ ]:
sentence = "[MASK] у сваім новым рамане «Ноч», які пабачыць свет на пачатку восені, прапаноўвае сваю версію таго, што зробіцца з нашай краінай, калі такое адбудзецца на самай справе."
original_word = "Пісьменнік"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

пісьменнік: 0.9950
літаратар: 0.9945
аўтар: 0.9916
паэт: 0.9888
стваральнік: 0.9841
мастак слова: 0.9798
журналіст: 0.9765
чытач: 0.9750
госць: 0.9722
сюжэт: 0.9719


In [ ]:
sentence = "Хочаце самастойна смела адкрываць візы, хутка шукаць [MASK] авіяквіткі, атрымліваць навіны пра распродажы авіякампаніяў?"
original_word = "танныя"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

танныя: 1.0000
дарагія: 0.9960
недарагі: 0.9932
даступны: 0.9890
простыя: 0.9867
сабе: 0.9860
гандлёвыя: 0.9859
бясплатныя: 0.9859
даступныя: 0.9852
рэдкія: 0.9849


In [ ]:
sentence = "Ён гатовы [MASK] у гэтым ключы любыя праблемы, што ўзніклі."
original_word = "абмеркаваць"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

абмеркаваць: 1.0000
абмяркоўваць: 0.9934
абгаварыць: 0.9904
абдумаць: 0.9581
разглядаць: 0.9487
разгледзець: 0.9484
вырашаць: 0.9463
выказваць: 0.9443
выказаць: 0.9406
вырашыць: 0.9403


In [ ]:
sentence = "Ён [MASK] пакуты, ён супраціўляўся сьмерці."
original_word = "цярпеў"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

пацярпеў: 0.9960
зазнаваць: 0.9867
праходзіў: 0.9827
пражыў: 0.9827
трымаў: 0.9806
атрымаў: 0.9806
правёў: 0.9802
прайшоў: 0.9797
атрымліваў: 0.9777
насіў: 0.9761


In [ ]:
sentence = "У Менску [MASK] адкрылі памятны знак у гонар братоў Луцкевічаў."
original_word = "ўрачыста"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

ўрачыста: 1.0000
урачыста: 0.9949
святочна: 0.9884
па-святочнаму: 0.9866
велічна: 0.9815
масава: 0.9814
годна: 0.9794
шырока: 0.9792
афіцыйна: 0.9770
спас: 0.9701


In [ ]:
sentence = "Юнакі і дзяўчаты не павінны [MASK] ставіцца да таго, што адбываецца ў краіне."
original_word = "абыякава"

predicted_tokens = get_masked_predictions(sentence, original_word, synonym_dict=synonym_dict)
ranked_suggestions = rank_by_similarity(sentence, predicted_tokens, original_word)

for word, similarity in ranked_suggestions[:10]:
    print(f"{word}: {similarity:.4f}")

абыякава: 1.0000
безудзельна: 0.9898
безуважна: 0.9886
незацікаўлена: 0.9866
бестурботна: 0.9864
бесклапотна: 0.9854
бесцікаўна: 0.9833
адчужана: 0.9823
няўцямна: 0.9800
бясстрасна: 0.9799
